# Example Python workflow for shuffling the climate data with tests to check whether shuffling worked

This notebook shows how you can shuffle the climate data (here for simplicity done on yearly basis). In addition, it produces test files of shuffled climate data for two glaciers (using nearest gridpoints of these glaciers). 

we choose these glaciers for testing:
- **RGI60-11.00897**: Hintereisferner (lon: 10.758, lat: 46.800)
    - nearest gridpoint from isimip3b: (10.75, 46.75)
- **RGI60-16.02207**: Shallap Glacier (lon: -9.486, lat: -77.334)
    - nearest gridpoint from isimip3b: (-9.25, -77.25)

**Please check in your workflow if your shuffling works by testing if you get the same annual time series of shuffled climate time series for the two glaciers. Note: no weighting per month duration is performed for the annual mean)!**

(we only check temperature shuffling with the ssp585 scenario and the ipsl-cm6a-lr gcm)

test files for shuffled climate:
- `test_shuffling/test_RGI60-11.00897_ipsl-cm6a-lr_ssp585_tasAdjust_shuffled.csv`
- `test_shuffling/test_RGI60-16.02207_ipsl-cm6a-lr_ssp585_tasAdjust_shuffled.csv`
---

In [1]:
# import these packages 
import xarray as xr
import numpy as np
import pandas as pd

let's take the *ipsl-cm6a-lr* gcm, the *ssp585* scenario and temperature as an example!

In [2]:
gcm = 'ipsl-cm6a-lr'
scenario = 'ssp585'
typ = 'tasAdjust'
glacier = 'RGI60-11.00897' 
# we also run the workflow for the Shallap glacier
# just run the notebook instead with:
# glacier = 'RGI60-16.02207'

In [3]:
# take the right gridpoint!
if glacier == 'RGI60-11.00897':
    # Hintereisferner
    lon, lat = (10.758, 46.800)
elif glacier == 'RGI60-16.02207':
    # Shallap glacier
    lon, lat = (-9.486, -77.334)

In [4]:
# get the shuffled year key:
pd_shuffled_yrs = pd.read_csv('shuffled_years_GlacierMIP3.csv', index_col=0)
pd_shuffled_yrs

,1851-1870,1901-1920,1951-1970,1995-2014,2021-2040,2041-2060,2061-2080,2081-2100
simulation_years,,,,,,,,
0,1861,1909,1957,2000,2034,2049,2073,2081
1,1858,1918,1959,2013,2036,2045,2072,2094
2,1852,1903,1969,2005,2032,2054,2065,2098
3,1862,1906,1963,1996,2039,2053,2061,2084
4,1868,1904,1970,2010,2021,2043,2063,2090
...,...,...,...,...,...,...,...,...
4995,1867,1905,1962,2008,2024,2053,2078,2088
4996,1852,1907,1968,2005,2027,2052,2073,2081
4997,1866,1913,1967,2012,2026,2049,2067,2093


In [5]:
# template of shuffled climate values (that will be filled afterwards)
periods = pd_shuffled_yrs.columns
simulation_years = pd_shuffled_yrs.index  # from 0 to 4999
pd_empty_clim_template = pd.DataFrame(np.NaN, columns=periods, index=simulation_years)
pd_empty_clim_template

,1851-1870,1901-1920,1951-1970,1995-2014,2021-2040,2041-2060,2061-2080,2081-2100
simulation_years,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
4995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# open the right climate file
if gcm in ['gfdl-esm4', 'ipsl-cm6a-lr', 'mpi-esm1-2-hr', 'mri-esm2-0']:
    ensemble = 'r1i1p1f1'
elif gcm == 'ukesm1-0-ll':
    ensemble = 'r1i1p1f2'

folder_output = f'isimip3b_{typ}_monthly'

# historical dataset
path_output_tas_hist = f'isimip3b/{folder_output}/{gcm}_{ensemble}_w5e5_historical_{typ}_global_monthly_1850_2014.nc'
ds_tas_monthly_hist = xr.open_dataset(path_output_tas_hist)

# ssp dataset
path_output_tas_ssp = f'isimip3b/{folder_output}/{gcm}_{ensemble}_w5e5_{scenario}_{typ}_global_monthly_2015_2100.nc'
ds_tas_monthly_ssp = xr.open_dataset(path_output_tas_ssp)

FileNotFoundError: [Errno 2] No such file or directory: b'/home/mowglie/disk/Dropbox/HomeDocs/git/GlacierMIP3/shuffling/isimip3b/isimip3b_tasAdjust_monthly/ipsl-cm6a-lr_r1i1p1f1_w5e5_historical_tasAdjust_global_monthly_1850_2014.nc'

In [7]:
# select the nearest grid point and get the annual means
# (note: no weighting per month duration is performed for the annual mean)
ds_yearly_hist = ds_tas_monthly_hist.sel(lon=lon, lat=lat, method='nearest').tasAdjust.groupby('time.year').mean()
ds_yearly_ssp = ds_tas_monthly_ssp.sel(lon=lon, lat=lat, method='nearest').tasAdjust.groupby('time.year').mean()
# concat historical with ssp file
ds_yearly_clim = xr.concat([ds_yearly_hist, ds_yearly_ssp], dim='year')

now we do the shuffling:

In [8]:
pd_shuffle_clim = pd_empty_clim_template.copy()
# get the shuffled climate data for each experiment (time period)
for p in periods:
    pd_shuffle_clim[p] = ds_yearly_clim.sel(year=pd_shuffled_yrs[p].values).values

# test file to check for your workflow
pd_shuffle_clim.to_csv(f'test_shuffling/test_{glacier}_{gcm}_{scenario}_{typ}_shuffled.csv')
pd_shuffle_clim

,1851-1870,1901-1920,1951-1970,1995-2014,2021-2040,2041-2060,2061-2080,2081-2100
simulation_years,,,,,,,,
0,270.509644,271.286499,271.965332,272.543823,273.897675,275.207062,278.033691,278.575348
1,271.433563,270.225555,272.337372,273.612549,274.926422,275.436096,278.030304,279.327179
2,271.823822,269.993378,271.553131,273.208374,274.084625,275.777252,277.347626,281.086548
3,271.572571,271.005493,271.319458,272.492340,274.388916,274.980591,277.015839,278.625946
4,271.465179,270.070770,270.637848,272.844513,273.681488,275.692261,276.637665,280.027466
...,...,...,...,...,...,...,...,...
4995,270.676910,269.957367,272.018311,272.476440,272.553009,274.980591,278.677673,279.783203
4996,271.823822,269.597382,270.914856,273.208374,273.274567,275.753815,278.033691,278.575348
4997,269.981049,270.118652,272.242859,273.874298,273.564636,275.207062,276.350616,280.937592
